In [1]:
import pandas as pd
import random
import json
import csv

with open('singgalang_ambigu.json', encoding="utf8") as f:
    listt = json.load(f)

In [2]:
list_tmp = []
for i in listt: 
    list_tmp.append(i.get("doc_id"))

In [3]:
list_tmp = [i.replace("Singgalang#","") for i in list_tmp]

In [4]:
list_sentence = []
nomor_sentence = 1

# open file in read mode
with open('SINGGALANG2.tsv', encoding='UTF-8') as read_obj:
    # pass the file object to reader() to get the reader object
    csv_reader = csv.reader(read_obj, delimiter='\t', quotechar=None) #https://www.petanikode.com/python-csv/ 
# https://docs.python.org/id/3.9/library/csv.html
# Menginstruksikan reader untuk tidak melakukan pemrosesan khusus terhadap karakter kutipan
    # Iterate over each row in the csv using reader object
    for row in csv_reader:
        # row variable is a list that represents a row in csv
        if len(row) == 0:
            nomor_sentence += 1 
        else:
            tmp_row= row
            tmp_row.append(nomor_sentence)
            list_sentence.append(tmp_row)

dataset = pd.DataFrame(list_sentence, columns=['token', 'ne', 'sentence'])

In [5]:
df_test = dataset[dataset['sentence'].isin(list_tmp)]
df_test = df_test.reset_index(drop=True)

In [6]:
df_train = dataset[~dataset['sentence'].isin(list_tmp)]
df_train = df_train.reset_index(drop=True)

In [7]:
def append_nan_rows(df_in):
    '''
    Menambahkan baris NaN di antara baris token terakhir
    suatu kalimat dan baris tokem awal kalimat selanjutnya.
    '''
    list_tmp = []
    for i in df_in.itertuples():
        if (i.Index > 0):
            if (df_in.iloc[i.Index].sentence != \
                df_in.iloc[i.Index-1].sentence):
                list_tmp.append([float("NaN"),
                                 float("NaN"),
                                 float("NaN")])
                list_tmp.append([i.token, i.ne, i.sentence])
            else:
                list_tmp.append([i.token, i.ne, i.sentence])
        else:
            list_tmp.append([i.token, i.ne, i.sentence])
    return df_in

In [8]:
def save_dataframe(df_in, str_fname):
    '''
    Menyimpan DataFrame dalam sebuah berkas TSV.
    '''
    try:
        df_in = df_in.drop("sentence", axis=1)
        df_in.to_csv(str_fname, sep="\t",
                      header=None,
                      index=False)
        print("DataFrame berhasil disimpan dalam", str_fname)
    except Exception as e:
        print(e)

In [9]:
df_train_new = append_nan_rows(df_train)
df_test_new = append_nan_rows(df_test)

save_dataframe(df_train_new, "singgalang_train.tsv")
save_dataframe(df_test_new, "singgalang_test.tsv")

DataFrame berhasil disimpan dalam singgalang_train.tsv
DataFrame berhasil disimpan dalam singgalang_test.tsv


---